In [1]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam
from google.colab import files
from keras.applications.inception_v3 import InceptionV3
from keras import optimizers
from keras.optimizers import Adam, SGD , RMSprop


Using TensorFlow backend.


In [7]:
!pip install -U -q kaggle
!mkdir  /root/.kaggle

files.upload()

mkdir: cannot create directory ‘/root/.kaggle’: File exists


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"isbincapps","key":"732d557bead1d64a89650c03d51b38cf"}'}

In [8]:
!cp kaggle.json /root/.kaggle
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

 99% 1.14G/1.15G [00:20<00:00, 44.3MB/s]
100% 1.15G/1.15G [00:20<00:00, 61.6MB/s]


In [0]:
!apt install pv
!unzip -o /content/chest-xray-pneumonia.zip  | pv -l >/dev/null
os.remove('chest-xray-pneumonia.zip')
!unzip -o /content/chest_xray.zip  | pv -l >/dev/null
os.remove('chest_xray.zip')

In [26]:

# batch_size = 32

# target_size = (299, 299)
# color_mode = "rgb"


rescale = 1./255
target_size = (150, 150)
batch_size = 163
class_mode = "categorical"
# class_mode = "binary"
train_dir = '/content/chest_xray/train'
test_dir = '/content/chest_xray/test'
val_dir = '/content/chest_xray/val'

def dir_file_count(directory):
    return sum([len(files) for r, d, files in os.walk(directory)])

train_datagen = ImageDataGenerator(
    rescale=rescale,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)


train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size,
    shuffle=True)


validation_datagen = ImageDataGenerator(rescale=rescale)

validation_generator = validation_datagen.flow_from_directory(
    val_dir,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=dir_file_count(val_dir),
    shuffle = False)

test_datagen = ImageDataGenerator(rescale=rescale)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=dir_file_count(test_dir),
    shuffle = False)

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [0]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

x = base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(2,activation='softmax')(x) #final layer with softmax activation

In [0]:
model=Model(inputs=base_model.input,outputs=preds)

for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True

In [36]:
#train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies

#train_generator=train_datagen.flow_from_directory('/content/chest_xray/train/',
#                                                 target_size=(150,150),
#                                                  color_mode='rgb',
#                                                  batch_size=32,
#                                                  class_mode='categorical', shuffle=True)
steps_per_epoch=len(train_generator)
validation_steps=len(validation_generator)

optimizer=optimizers.Adam()
loss='categorical_crossentropy'
metrics=['accuracy']

model.compile(optimizer, loss=loss, metrics=metrics)
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy

#step_size_train=train_generator.n//train_generator.batch_size
#model.fit_generator(generator=train_generator,
#                   steps_per_epoch=step_size_train,
#                   epochs=100)

model.fit_generator(
    train_generator,
    steps_per_epoch = steps_per_epoch,
    epochs=80,
    verbose=2,
    validation_data=validation_generator,
    validation_steps=validation_steps)

Epoch 1/80
 - 90s - loss: 0.0592 - acc: 0.9797 - val_loss: 5.6146 - val_acc: 0.5000
Epoch 2/80
 - 70s - loss: 0.0446 - acc: 0.9826 - val_loss: 4.5802 - val_acc: 0.5000
Epoch 3/80
 - 73s - loss: 0.0382 - acc: 0.9845 - val_loss: 5.5546 - val_acc: 0.5000
Epoch 4/80
 - 73s - loss: 0.0385 - acc: 0.9856 - val_loss: 4.5505 - val_acc: 0.5000
Epoch 5/80
 - 73s - loss: 0.0360 - acc: 0.9896 - val_loss: 5.3416 - val_acc: 0.5000
Epoch 6/80
 - 73s - loss: 0.0402 - acc: 0.9868 - val_loss: 3.7778 - val_acc: 0.5000
Epoch 7/80
 - 72s - loss: 0.0345 - acc: 0.9879 - val_loss: 5.7557 - val_acc: 0.5000
Epoch 8/80
 - 72s - loss: 0.0319 - acc: 0.9881 - val_loss: 5.6506 - val_acc: 0.5000
Epoch 9/80
 - 72s - loss: 0.0279 - acc: 0.9908 - val_loss: 5.6591 - val_acc: 0.5000
Epoch 10/80
 - 72s - loss: 0.0275 - acc: 0.9902 - val_loss: 5.8838 - val_acc: 0.5000
Epoch 11/80
 - 71s - loss: 0.0299 - acc: 0.9891 - val_loss: 4.5472 - val_acc: 0.5000
Epoch 12/80
 - 71s - loss: 0.0275 - acc: 0.9902 - val_loss: 5.3551 - val_a

In [0]:
model.save('chest-xray-pneumonia.h5')    


In [0]:
from keras.models import load_model
new_model = load_model("/content/chest-xray-pneumonia.h5")


In [39]:
result  = new_model.evaluate_generator(test_generator, steps=len(test_generator), verbose=1)
print("%s%.2f  "% ("Loss     : ", result[0]))
print("%s%.2f%s"% ("Accuracy : ", result[1]*100, "%"))

1/1 [==============================] - 17s 17s/step
Loss     : 4.50  
Accuracy : 63.14%


In [0]:
def get_rez(pic):
  img = image.load_img(pic, target_size=(150,150))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)
  prediction = new_model.predict(x)
  print('la prediction f', prediction)
  p_good,p_ill = np.around(prediction, decimals=2)[0]
  return{'p_good':p_good,'p_ill':p_ill}

In [40]:
#from google.colab import drive
#img = drive.mount('/content/chest_xray/test/NORMAL/IM-0001-0001.jpeg')
predict1 = get_rez('/content/chest_xray/val/NORMAL/NORMAL2-IM-1442-0001.jpeg')
predict2 = get_rez('/content/chest_xray/val/PNEUMONIA/person1946_bacteria_4875.jpeg')
print('la prediction 1', predict1)
print('la prediction 2', predict2)

la prediction f [[0.03923794 0.9607621 ]]
la prediction f [[8.24305e-09 1.00000e+00]]
la prediction 1 {'p_good': 0.04, 'p_ill': 0.96}
la prediction 2 {'p_good': 0.0, 'p_ill': 1.0}
